# AMEY ANANT PATIL

### Reddit BTC Sentiments

A 5KK database of Reddit Commentaries from Big Query is processed and the sentiments polarities are calculated with VADER and the aggregated per day.


***
## Necessary libs

In [4]:
pip install swifter


Note: you may need to restart the kernel to use updated packages.


In [2]:
import glob
import pandas as pd # usaremos o pandas com processamento paralelo do modin
import numpy as np
import fasttext
import unicodedata
import plotly.express as px
import re
import string
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.preprocessing import MinMaxScaler
import tqdm
from datetime import datetime
import swifter

In [5]:
pwd

'C:\\Users\\NATHDWAR\\Desktop\\Fitch Python\\Final project\\CQF_final_project-main'

In [7]:
    reddit = pd.DataFrame()
    reddit = pd.read_csv('part_1.csv')

***
## Functions

In [6]:
pwd

'C:\\Users\\NATHDWAR\\Desktop\\Fitch Python\\Final project\\CQF_final_project-main'

In [7]:
# Training Fast Text in a example database:
model = fasttext.load_model('lid.176.ftz')


In [8]:
# Fast Text is a Facebook NLP lib.


# Training Fast Text in a example database:
model = fasttext.load_model('lid.176.ftz')

# Function to identify language (VADER only works for english):

def fast_detect(msg):
    try:
        # The following predict returns the text language
        ln = model.predict(msg)[0][0].split("__")[2] 
    except:
        # It defaults to None when it fails
        ln = None
    return ln

In [9]:
# Function to remove special characters from comments
def convert_text(text):
    
    text = re.sub(r'\s+', ' ',str(text))
    
    
    text = re.sub(' +', ' ', text)
    
    text = text.replace("\\'","'")
    text = text.replace('\\','')
    
    text = text.replace('*','')
    text = text.replace('_','')

    
    text = re.sub(r'http\S+', '', text)
    

    return text

In [10]:
# Function to use VADER:
def VADER(text):
    
    sent_an = SentimentIntensityAnalyzer()
    
    
    sentiment_dict = sent_an.polarity_scores(text)

    # only the compound polarity score is used:
    return sentiment_dict['compound']

***
## Importing downloaded data

As the query retrieved a huge dataset, BigQuery partitioned it into multiple files to export.

In [11]:

reddit = pd.DataFrame()
for file in glob.glob('bitcoin_reddit_all*'):
    df = pd.read_csv(file)
    print(file,df.shape)
    reddit=reddit.append(df)

C:\Users\NATHDWAR\AppData\Local\Temp\ipykernel_10580\3791039482.py:3: DtypeWarning: Columns (0,1,2,3,4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


bitcoin_reddit_all.csv (4309244, 9)


C:\Users\NATHDWAR\AppData\Local\Temp\ipykernel_10580\3791039482.py:3: DtypeWarning: Columns (0,1,2,3,4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


bitcoin_reddit_all.csv.zip (4309244, 9)


Checking the file size:

In [12]:
reddit.shape #8618488

(8618488, 9)

The dataset comprises 8,618,488 commentaries mentioning either bitcoin or btc.

In [13]:
reddit.sample(1)
#reddit.sample(min(3, len(reddit)))


,Unnamed: 0,datetime,date,author,subreddit,created_utc,score,controversiality,body
2767174,2651295,2017-07-29 10:54:18,2017-07-29,maroule,Ripple,1.501326e+09,2.0,0.0,"Honestly, I have a hard time finding something..."


The table returned is structured as follows:

* subreddit

* UTC created date

* commentary body of text

* score = upvotes - downvotes (liquid score)

***
## Data processing

In [14]:
print(reddit.columns)


Index(['Unnamed: 0', 'datetime', 'date', 'author', 'subreddit', 'created_utc',
       'score', 'controversiality', 'body'],
      dtype='object')


In [15]:
# Converting from unix to UTC

reddit['Created Date'] = pd.to_datetime(reddit['created_utc'],unit = 's')

In [16]:
# Ordering by created date

reddit.sort_values('Created Date',inplace=True)

We use the previously defined function to remove special characters.

Due to the dataset size, **Swifter** was used for parallel processing.

In [17]:
reddit.head()

,Unnamed: 0,datetime,date,author,subreddit,created_utc,score,controversiality,body,Created Date
124711,124689,2009-05-08 13:54:29,2009-05-08,maurice,business,1.241791e+09,1.0,0.0,"Interesting, it uses IRC as a high level proto...",2009-05-08 13:54:29
124711,124689,2009-05-08 13:54:29,2009-05-08,maurice,business,1.241791e+09,1.0,0.0,"Interesting, it uses IRC as a high level proto...",2009-05-08 13:54:29
136797,136769,2009-05-08 14:29:56,2009-05-08,badassumption,business,1.241793e+09,2.0,0.0,No - the richest person will be the one with t...,2009-05-08 14:29:56
136797,136769,2009-05-08 14:29:56,2009-05-08,badassumption,business,1.241793e+09,2.0,0.0,No - the richest person will be the one with t...,2009-05-08 14:29:56
167844,167813,2009-05-08 16:35:36,2009-05-08,Enginerd,business,1.241801e+09,-3.0,0.0,&gt;a public list of all the previous transact...,2009-05-08 16:35:36


After removing special characters, the body language is classified with Fast Text:

In [18]:
# Removing special characters
reddit['Comment'] = reddit['body'].swifter.apply(convert_text)


# Infering language
reddit['Language'] = reddit['Comment'].swifter.apply(fast_detect)

C:\ProgramData\Anaconda3\lib\site-packages\swifter\swifter.py:87: UserWarning: This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`.
  warnings.warn(


Pandas Apply:   0%|          | 0/8618488 [00:00<?, ?it/s]

C:\ProgramData\Anaconda3\lib\site-packages\swifter\swifter.py:87: UserWarning: This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`.
  warnings.warn(


Pandas Apply:   0%|          | 0/8618488 [00:00<?, ?it/s]

In [25]:
reddit.head

<bound method NDFrame.head of         Unnamed: 0             datetime        date         author subreddit  \
124711      124689  2009-05-08 13:54:29  2009-05-08        maurice  business   
124711      124689  2009-05-08 13:54:29  2009-05-08        maurice  business   
136797      136769  2009-05-08 14:29:56  2009-05-08  badassumption  business   
136797      136769  2009-05-08 14:29:56  2009-05-08  badassumption  business   
167844      167813  2009-05-08 16:35:36  2009-05-08       Enginerd  business   
...            ...                  ...         ...            ...       ...   
3827061    3711178  2019-12-31 23:57:54  2019-12-31       femstora     4chan   
4231225    4115341  2019-12-31 23:58:07  2019-12-31   Benjamincito   Bitcoin   
4231225    4115341  2019-12-31 23:58:07  2019-12-31   Benjamincito   Bitcoin   
4256552    4140668  2019-12-31 23:58:38  2019-12-31  Skullpopper69  Buttcoin   
4256552    4140668  2019-12-31 23:58:38  2019-12-31  Skullpopper69  Buttcoin   

         

In [26]:
reddit.to_csv('part_1.csv')

In [28]:
#  Languages
cont = reddit['Language'].value_counts().to_frame('contagem')
cont2 = pd.DataFrame({'english':cont.loc['en'], 'others':cont.iloc[1:].sum(axis=0)})
cont2


,english,others
contagem,8404402,0.0


As VADER only works for english text, the other languages are removed.

In [29]:
reddit.query('Language == "en"',inplace=True)

VADER is then used to obtain the compound sentiment polarity score for each comment.

In [31]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import threading

# Download the VADER lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer after downloading the lexicon
VADER = SentimentIntensityAnalyzer()

# Define a function to process a chunk and save it to CSV
def process_chunk(chunk, index):
    chunk['Sentiment'] = chunk['Comment'].apply(VADER.polarity_scores)
    chunk.to_csv(f'bup{index}.csv', index=False)

# Assuming you have already loaded your Reddit data into the 'reddit' DataFrame

# Calculate the chunk size (10% of total rows)
chunk_size = len(reddit) // 10

# Create and start threads for processing chunks
threads = []
for i in range(10):
    start_idx = i * chunk_size
    end_idx = start_idx + chunk_size
    
    # Create a thread for each chunk
    chunk = reddit.iloc[start_idx:end_idx]
    thread = threading.Thread(target=process_chunk, args=(chunk, i))
    thread.start()
    threads.append(thread)

# Wait for all threads to finish
for thread in threads:
    thread.join()

print("All threads have finished processing.")


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\NATHDWAR\AppData\Roaming\nltk_data...
C:\Users\NATHDWAR\AppData\Local\Temp\ipykernel_10580\4063223723.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['Sentiment'] = chunk['Comment'].apply(VADER.polarity_scores)
C:\Users\NATHDWAR\AppData\Local\Temp\ipykernel_10580\4063223723.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['Sentiment'] = chunk['Comment'].apply(VADER.polarity_scores)
C:\Users\NATHDWAR\AppData\Local\

All threads have finished processing.


#VADER = SentimentIntensityAnalyzer()

# Calculate the chunk size (10% of total rows)
chunk_size = len(reddit) // 10

# Process the data in chunks
for i in range(10):
    start_idx = i * chunk_size
    end_idx = start_idx + chunk_size
    
    # Process the current chunk using VADER
    chunk = reddit.iloc[start_idx:end_idx]
    chunk['Sentiment'] = chunk['Comment'].apply(VADER.polarity_scores)
    
    # Export the chunk to a CSV file
    chunk.to_csv(f'bup{i}.csv', index=False)

total_rows = len(reddit)
chunk_size = ceil(total_rows / 10)

# Process the data in chunks
for i in tqdm(range(0, total_rows, chunk_size)):
    chunk = reddit.iloc[i:i+chunk_size]
    chunk['Sentiment'] = chunk['Comment'].apply(VADER.polarity_scores)
    # Process chunk further if needed
    
    # For example, you can save the processed chunk back to your DataFrame
    reddit.iloc[i:i+chunk_size] = chunk

#Once again using swifter to accelearate processing:
reddit['Sentiment'] = reddit['Comment'].swifter.apply(VADER,axis=0)



reddit.to_csv('bup1.csv') #saving a backup


The scores are normalized with MinMaxScaler (there are negative scores) and used to weight the compound polarity.

In [47]:
df2 = pd.read_csv('bup0.csv')
#reddit3=reddit3.append(df)
df2.head()

,Unnamed: 0,datetime,date,author,subreddit,created_utc,score,controversiality,body,Created Date,Comment,Language,Sentiment
0,124689,2009-05-08 13:54:29,2009-05-08,maurice,business,1.241791e+09,1.0,0.0,"Interesting, it uses IRC as a high level proto...",2009-05-08 13:54:29,"Interesting, it uses IRC as a high level proto...",en,"{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'comp..."
1,124689,2009-05-08 13:54:29,2009-05-08,maurice,business,1.241791e+09,1.0,0.0,"Interesting, it uses IRC as a high level proto...",2009-05-08 13:54:29,"Interesting, it uses IRC as a high level proto...",en,"{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'comp..."
2,136769,2009-05-08 14:29:56,2009-05-08,badassumption,business,1.241793e+09,2.0,0.0,No - the richest person will be the one with t...,2009-05-08 14:29:56,No - the richest person will be the one with t...,en,"{'neg': 0.09, 'neu': 0.772, 'pos': 0.138, 'com..."
3,136769,2009-05-08 14:29:56,2009-05-08,badassumption,business,1.241793e+09,2.0,0.0,No - the richest person will be the one with t...,2009-05-08 14:29:56,No - the richest person will be the one with t...,en,"{'neg': 0.09, 'neu': 0.772, 'pos': 0.138, 'com..."
4,167813,2009-05-08 16:35:36,2009-05-08,Enginerd,business,1.241801e+09,-3.0,0.0,&gt;a public list of all the previous transact...,2009-05-08 16:35:36,&gt;a public list of all the previous transact...,en,"{'neg': 0.115, 'neu': 0.781, 'pos': 0.104, 'co..."


In [55]:
df2.head()

,Unnamed: 0,datetime,date,author,subreddit,created_utc,score,controversiality,body,Created Date,Comment,Language,Sentiment,Compound_Score
0,124689,2009-05-08 13:54:29,2009-05-08,maurice,business,1.241791e+09,1.0,0.0,"Interesting, it uses IRC as a high level proto...",2009-05-08 13:54:29,"Interesting, it uses IRC as a high level proto...",en,"{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'comp...",0.4019
1,124689,2009-05-08 13:54:29,2009-05-08,maurice,business,1.241791e+09,1.0,0.0,"Interesting, it uses IRC as a high level proto...",2009-05-08 13:54:29,"Interesting, it uses IRC as a high level proto...",en,"{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'comp...",0.4019
2,136769,2009-05-08 14:29:56,2009-05-08,badassumption,business,1.241793e+09,2.0,0.0,No - the richest person will be the one with t...,2009-05-08 14:29:56,No - the richest person will be the one with t...,en,"{'neg': 0.09, 'neu': 0.772, 'pos': 0.138, 'com...",0.5267
3,136769,2009-05-08 14:29:56,2009-05-08,badassumption,business,1.241793e+09,2.0,0.0,No - the richest person will be the one with t...,2009-05-08 14:29:56,No - the richest person will be the one with t...,en,"{'neg': 0.09, 'neu': 0.772, 'pos': 0.138, 'com...",0.5267
4,167813,2009-05-08 16:35:36,2009-05-08,Enginerd,business,1.241801e+09,-3.0,0.0,&gt;a public list of all the previous transact...,2009-05-08 16:35:36,&gt;a public list of all the previous transact...,en,"{'neg': 0.115, 'neu': 0.781, 'pos': 0.104, 'co...",-0.0772


In [56]:
def extract_compound_score(sentiment_dict_str):
    compound_score_match = re.search(r"'compound':\s*(-?\d+\.\d+)", sentiment_dict_str)
    if compound_score_match:
        return float(compound_score_match.group(1))
    else:
        return None

# Apply the function to create a new column 'Compound_Score'
df2['Compound_Score'] = (df2['Sentiment'].apply(extract_compound_score))
#reddit1 = reddit1[['Compound_Score']]
# Print the result
print(df2.head())


   Unnamed: 0             datetime        date         author subreddit  \
0      124689  2009-05-08 13:54:29  2009-05-08        maurice  business   
1      124689  2009-05-08 13:54:29  2009-05-08        maurice  business   
2      136769  2009-05-08 14:29:56  2009-05-08  badassumption  business   
3      136769  2009-05-08 14:29:56  2009-05-08  badassumption  business   
4      167813  2009-05-08 16:35:36  2009-05-08       Enginerd  business   

    created_utc  score  controversiality  \
0  1.241791e+09    1.0               0.0   
1  1.241791e+09    1.0               0.0   
2  1.241793e+09    2.0               0.0   
3  1.241793e+09    2.0               0.0   
4  1.241801e+09   -3.0               0.0   

                                                body         Created Date  \
0  Interesting, it uses IRC as a high level proto...  2009-05-08 13:54:29   
1  Interesting, it uses IRC as a high level proto...  2009-05-08 13:54:29   
2  No - the richest person will be the one with t... 

In [64]:
empty_columns = ['Unnamed: 0', 'datetime', 'date', 'author', 'subreddit', 'created_utc', 'score',
                 'controversiality', 'body', 'Created Date', 'Comment', 'Language', 'Sentiment']
empty_data = pd.DataFrame(columns=empty_columns)

# Overwrite the 'reddit' DataFrame with the new empty DataFrame
reddit = empty_data
df3=empty_data
# Print the empty DataFrame
print(reddit)

Empty DataFrame
Columns: [Unnamed: 0, datetime, date, author, subreddit, created_utc, score, controversiality, body, Created Date, Comment, Language, Sentiment]
Index: []


In [65]:

for file in glob.glob('bup*'):
    df3 = pd.read_csv(file)
    print(file,df.shape)
    reddit=reddit.append(df3)

bup0.csv (840440, 13)
bup1.csv (840440, 13)
bup2.csv (840440, 13)
bup3.csv (840440, 13)
bup4.csv (840440, 13)
bup5.csv (840440, 13)
bup6.csv (840440, 13)
bup7.csv (840440, 13)
bup8.csv (840440, 13)
bup9.csv (840440, 13)


In [66]:
#reddit2 = reddit
reddit.head()

,Unnamed: 0,datetime,date,author,subreddit,created_utc,score,controversiality,body,Created Date,Comment,Language,Sentiment
0,124689,2009-05-08 13:54:29,2009-05-08,maurice,business,1.241791e+09,1.0,0.0,"Interesting, it uses IRC as a high level proto...",2009-05-08 13:54:29,"Interesting, it uses IRC as a high level proto...",en,"{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'comp..."
1,124689,2009-05-08 13:54:29,2009-05-08,maurice,business,1.241791e+09,1.0,0.0,"Interesting, it uses IRC as a high level proto...",2009-05-08 13:54:29,"Interesting, it uses IRC as a high level proto...",en,"{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'comp..."
2,136769,2009-05-08 14:29:56,2009-05-08,badassumption,business,1.241793e+09,2.0,0.0,No - the richest person will be the one with t...,2009-05-08 14:29:56,No - the richest person will be the one with t...,en,"{'neg': 0.09, 'neu': 0.772, 'pos': 0.138, 'com..."
3,136769,2009-05-08 14:29:56,2009-05-08,badassumption,business,1.241793e+09,2.0,0.0,No - the richest person will be the one with t...,2009-05-08 14:29:56,No - the richest person will be the one with t...,en,"{'neg': 0.09, 'neu': 0.772, 'pos': 0.138, 'com..."
4,167813,2009-05-08 16:35:36,2009-05-08,Enginerd,business,1.241801e+09,-3.0,0.0,&gt;a public list of all the previous transact...,2009-05-08 16:35:36,&gt;a public list of all the previous transact...,en,"{'neg': 0.115, 'neu': 0.781, 'pos': 0.104, 'co..."


In [67]:
def extract_compound_score(sentiment_dict_str):
    compound_score_match = re.search(r"'compound':\s*(-?\d+\.\d+)", sentiment_dict_str)
    if compound_score_match:
        return float(compound_score_match.group(1))
    else:
        return None

# Apply the function to create a new column 'Compound_Score'
reddit['Compound_Score'] = (reddit['Sentiment'].apply(extract_compound_score))
#reddit1 = reddit1[['Compound_Score']]
# Print the result
print(reddit.head())

  Unnamed: 0             datetime        date         author subreddit  \
0     124689  2009-05-08 13:54:29  2009-05-08        maurice  business   
1     124689  2009-05-08 13:54:29  2009-05-08        maurice  business   
2     136769  2009-05-08 14:29:56  2009-05-08  badassumption  business   
3     136769  2009-05-08 14:29:56  2009-05-08  badassumption  business   
4     167813  2009-05-08 16:35:36  2009-05-08       Enginerd  business   

    created_utc  score  controversiality  \
0  1.241791e+09    1.0               0.0   
1  1.241791e+09    1.0               0.0   
2  1.241793e+09    2.0               0.0   
3  1.241793e+09    2.0               0.0   
4  1.241801e+09   -3.0               0.0   

                                                body         Created Date  \
0  Interesting, it uses IRC as a high level proto...  2009-05-08 13:54:29   
1  Interesting, it uses IRC as a high level proto...  2009-05-08 13:54:29   
2  No - the richest person will be the one with t...  2009-

In [70]:
#reddit['Sentiment_All'] = reddit['Sentiment'].apply(lambda x: x['compound'])
#reddit.rename(columns={'Sentiment': 'Sentiment_All'}, inplace=True)
#reddit.rename(columns={'Compound_Score': 'Sentiment'}, inplace=True)
print(reddit.head())

  Unnamed: 0             datetime        date         author subreddit  \
0     124689  2009-05-08 13:54:29  2009-05-08        maurice  business   
1     124689  2009-05-08 13:54:29  2009-05-08        maurice  business   
2     136769  2009-05-08 14:29:56  2009-05-08  badassumption  business   
3     136769  2009-05-08 14:29:56  2009-05-08  badassumption  business   
4     167813  2009-05-08 16:35:36  2009-05-08       Enginerd  business   

    created_utc  score  controversiality  \
0  1.241791e+09    1.0               0.0   
1  1.241791e+09    1.0               0.0   
2  1.241793e+09    2.0               0.0   
3  1.241793e+09    2.0               0.0   
4  1.241801e+09   -3.0               0.0   

                                                body         Created Date  \
0  Interesting, it uses IRC as a high level proto...  2009-05-08 13:54:29   
1  Interesting, it uses IRC as a high level proto...  2009-05-08 13:54:29   
2  No - the richest person will be the one with t...  2009-

In [71]:
# Necessary Columns:
reddit = reddit[['subreddit','score','Created Date','Comment','Sentiment']]
reddit.head()

,subreddit,score,Created Date,Comment,Sentiment
0,business,1.0,2009-05-08 13:54:29,"Interesting, it uses IRC as a high level proto...",0.4019
1,business,1.0,2009-05-08 13:54:29,"Interesting, it uses IRC as a high level proto...",0.4019
2,business,2.0,2009-05-08 14:29:56,No - the richest person will be the one with t...,0.5267
3,business,2.0,2009-05-08 14:29:56,No - the richest person will be the one with t...,0.5267
4,business,-3.0,2009-05-08 16:35:36,&gt;a public list of all the previous transact...,-0.0772


In [72]:
reddit.duplicated(keep='first').sum()

4256522

Removing the exact duplicated observations:

In [73]:
reddit.drop_duplicates(inplace=True)
reddit.shape[0]

4147878

The dataset goes from 8404402 to 4147878 observations after removing duplicates.

In [74]:
# Checking vader performance:
print(reddit.query('Sentiment==1').sample(1)['Comment'].values)

["amazon gift card free, amazon gift card codes free, amazon gift card 2017, amazon gift card hack, amazon gift card generator, amazon gift card giveaway, amazon gift card code generator, amazon gift card giveaway live, amazon gift card generator 2017, amazon gift card app, amazon gift card, amazon gift card code, amazon gift card balance, amazon gift card api, amazon gift card ad, amazon gift card adder, amazon gift card app hack, amazon gift card android, amazon gift card at best buy, amazon gift card and promotional codes, amazon gift card activator, amazon gift card already redeemed, a amazon gift card code, amazon gift card buy, amazon gift card box, amazon gift card barcode, amazon gift card balance mobile recharge, amazon gift card balance transfer to bank account, amazon gift card balance hack, amazon gift card balance transfer, amazon gift card bing rewards, amazon gift card best buy, amazon gift card code giveaway, amazon gift card codes 2017, amazon gift card codes 2017 no s

In [75]:
print(reddit.query('Sentiment==-1').sample(1)['Comment'].values)

["The following comment by throwaway34234234523 was [openly]( greylisted. The original comment can be found(in censored form) at this link: np.reddit.com/r/ CryptoCurrency/comments/7poep7/-/dsire5o?context=4 The original comment's content was as follows: --- &gt; XRP is cancer for cryptocurrencies. Im sure you dont know abour technical specs of ripple, but is the worst coin for crypto. If it turns in main one day (as bitcoin), they will decrease the price of all alts until they cost almost zero, creating new coins. &gt; &gt; &gt;  100% Premine (cant mine new coins) &gt;  60% Held for creators (control) &gt;  No staking / No minig (no rewards for normal users) &gt;  Private blockchain (control, sell personal data) &gt;  Centralized (they can shutdown XRP blockchain when they want and crash the parity alts) &gt;  50$ stucked in wallet (20 XRP) (staking without reward) &gt;  Bound to identity (control, sell personal data) &gt; &gt; &gt; DO NOT SUPPORT RIPPLE (XRP), IT WANTS TO KILL CRYPTO

It is clear that the comments that scored either the maximum / minimum compound polarity (+1, -1) are spams.

This happens because VADER considers upper and lower cases as intensity indicator. Considering that only a few observations had the maximum/minimum score (3 observations), they are removed.


The other similar occurrences are controled by using the influence as weight (score).

In [76]:
reddit.query('Sentiment !=-1 & Sentiment != 1', inplace=True)

In [77]:
# Normalizing scores between 0-1

scaler = MinMaxScaler()

reddit['score_norm'] = scaler.fit_transform(reddit['score'].values.reshape(-1, 1))

# Normalized Scores * Compount Polarity

reddit['Sentiment_weighted'] = reddit['Sentiment']*(reddit['score_norm'])

# Thus, the weighted sentiment varies between -1 and 1.

In [78]:
# Defining daily dates:
reddit['Day'] = pd.to_datetime(reddit['Created Date']).dt.date

In [79]:
reddit.head()

,subreddit,score,Created Date,Comment,Sentiment,score_norm,Sentiment_weighted,Day
0,business,1.0,2009-05-08 13:54:29,"Interesting, it uses IRC as a high level proto...",0.4019,0.053031,0.021313,2009-05-08
2,business,2.0,2009-05-08 14:29:56,No - the richest person will be the one with t...,0.5267,0.053064,0.027949,2009-05-08
4,business,-3.0,2009-05-08 16:35:36,&gt;a public list of all the previous transact...,-0.0772,0.052901,-0.004084,2009-05-08
6,business,2.0,2009-05-08 19:35:24,"No, that's not how bitcoin works, check out th...",-0.2960,0.053064,-0.015707,2009-05-08
8,business,0.0,2009-07-18 21:38:06,It's weird how Ron Paul gets money so incredib...,0.4968,0.052999,0.026330,2009-07-18


The dataset is exported.

In [80]:
reddit.to_csv('reddit.csv')

In [81]:
reddit.shape

(4147875, 8)